# Evaluación de Eficiencia de Modelos

En este cuaderno se evalúa la eficiencia computacional de los modelos entrenados: **Random Forest**, **Regresión Logística** y **SVM**. Esta evaluación se centra en:

- Tiempo de inferencia (tiempo que tarda cada modelo en realizar una predicción)
- Capacidad de uso en tiempo real o entornos con recursos limitados
- Especificaciones del sistema utilizado para la evaluación

Este análisis complementa la evaluación métrica tradicional (F1-score, AUC) proporcionando una visión más práctica sobre el comportamiento de los modelos en producción.


## Carga de Modelos Entrenados

En este notebook se comparan los tres modelos entrenados previamente: **Random Forest**, **Regresión Logística** y **SVM (kernel RBF con subconjunto reducido)**.

Se cargan directamente desde los archivos `.pkl` guardados durante el entrenamiento, permitiendo reutilizarlos sin volver a entrenarlos. Esto garantiza reproducibilidad y eficiencia en la evaluación final.


In [2]:
import joblib

# Cargar modelos previamente guardados
rf_model = joblib.load("../models/random_forest_model.pkl")
lr_model = joblib.load("../models/logistic_regression_model.pkl")
svm_model = joblib.load("../models/svm_model.pkl")

print("Modelos cargados correctamente:")
print("- Random Forest")
print("- Logistic Regression")
print("- SVM (entrenado con 10,000 muestras)")


Modelos cargados correctamente:
- Random Forest
- Logistic Regression
- SVM (entrenado con 10,000 muestras)


In [4]:
from sklearn.metrics import f1_score, roc_auc_score

# Asegúrate de que tienes X_test e y_test cargados desde el dataset limpio
import pandas as pd

# Cargar el dataset limpio y dividir
df = pd.read_csv("../archive/cleaned_dataset.csv")
X = df.drop(columns=["Label"])
y = df["Label"]

# División en test para evaluación (20% del total, como antes)
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Obtener predicciones y probabilidades (cuando sea posible)
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

y_pred_lr = lr_model.predict(X_test)
y_prob_lr = lr_model.predict_proba(X_test)[:, 1]

y_pred_svm = svm_model.predict(X_test)
y_score_svm = svm_model.decision_function(X_test)  # no .predict_proba()

# Calcular métricas
resultados = pd.DataFrame({
    "Modelo": ["Random Forest", "Logistic Regression", "SVM (RBF, 10k)"],
    "F1-score": [
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_svm)
    ],
    "AUC": [
        roc_auc_score(y_test, y_prob_rf),
        roc_auc_score(y_test, y_prob_lr),
        roc_auc_score(y_test, y_score_svm)
    ]
})

# Redondear para mejor visualización
resultados["F1-score"] = resultados["F1-score"].round(4)
resultados["AUC"] = resultados["AUC"].round(4)

resultados


,Modelo,F1-score,AUC
0,Random Forest,0.9999,1.0000
1,Logistic Regression,0.9916,0.9992
2,"SVM (RBF, 10k)",0.8252,0.6713


## Análisis Comparativo de Métricas de Clasificación

La siguiente tabla muestra los resultados de evaluación obtenidos por los tres modelos entrenados y probados sobre el mismo conjunto de datos:

| Modelo              | F1-score | AUC   |
|---------------------|----------|-------|
| Random Forest       | 0.9999   | 1.0000 |
| Logistic Regression | 0.9916   | 0.9992 |
| SVM (RBF, 10k)      | 0.8252   | 0.6713 |

### Interpretación:

- **Random Forest** obtuvo resultados perfectos en ambas métricas, lo que indica una separación total entre clases. Esto se alinea con los resultados observados en la matriz de confusión y curva ROC, donde no se produjeron errores. Sin embargo, su interpretabilidad es menor que la de otros modelos.

- **Regresión Logística** también mostró un rendimiento prácticamente perfecto. Aunque es un modelo mucho más simple, logró un F1-score y AUC cercanos a 1.0. Este resultado refuerza la idea de que el dataset CICIDS2017 permite separar las clases incluso con modelos lineales bien ajustados. Además, su interpretabilidad y eficiencia computacional son ventajas destacadas.

- **SVM** (entrenado con un subconjunto de 10,000 muestras debido a restricciones de tiempo y coste computacional) obtuvo un rendimiento claramente inferior. El F1-score de 0.83 y AUC de 0.67 indican que, aunque el modelo es capaz de detectar algunos ataques, genera muchos falsos positivos. Este resultado es consistente con lo observado en la matriz de confusión y la curva ROC del modelo SVM. A pesar de su sensibilidad, su utilidad práctica se ve limitada sin ajustes adicionales.

### Conclusión:

Los resultados confirman que, para este dataset, los modelos Random Forest y Regresión Logística ofrecen una detección muy precisa del tráfico malicioso. La regresión logística, además, lo consigue con una estructura más sencilla y un coste computacional más bajo, lo que la convierte en una opción especialmente interesante cuando se requiere interpretabilidad o rapidez. SVM, aunque potente teóricamente, no resulta competitivo en este caso sin un mayor ajuste y entrenamiento completo.
